In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [2]:
def parse_minutes(x):
    
    spl = x.split('.')[0].split(' ')
    hours = int(spl[0]) * 60
    mins = int(spl[2]) if spl[2] != '' else 0
    return hours + mins

def transpose_data(df, col):
    
    df.set_index(col, inplace=True)
    df = df.transpose().reset_index()
    df['time'] = df['index'].apply(parse_minutes)
    df = df.set_index('time').drop('index', axis=1)
    return df

def generate_data(df, name, h=24, m=0, num_data=4, start_idx=3, col='short_name'):
    
    datas = []
    idxs = [start_idx]
    for i in range(1, num_data+1):
        idxs.append(h * 3 + idxs[i-1] + 1 + int(m/20))
        data = (df.iloc[:, idxs[i-1]:idxs[i]]).astype(float)
        data = pd.concat([name, data], axis=1)
        data = transpose_data(data, col)
        datas.append(data)
    return datas

def read_map(plate_map, ctrl='H'):
    
    plate_map = plate_map.melt(id_vars=['group'])
    plate_map['variable'] = plate_map['variable'].astype(int)
    plate_map['Well'] = plate_map['group'] + plate_map['variable'].apply(lambda x: '{:02d}'.format(x))
    plate_map.loc[~plate_map['group'].isin([ctrl]), 'suffix'] = (plate_map['variable'] - 1) % 6
    plate_map.loc[plate_map['group'].isin([ctrl]), 'suffix'] = (plate_map['variable'] - 1) % 2
    return plate_map[['Well', 'value', 'suffix']].dropna()

def generate_metadata(well, plate_map):
    
    dictionary = pd.read_csv('datasets/dictionary.csv')
    sample_map = {
        'P62': 'pBAD-rJ/Ara',
        'P63': 'araC-pBAD-rJ/Ara',
        'P64': 'pCin-rJ/OHC14',
        'P65': 'pCymRC-rJ/Cuma',
        'P66': 'cymRC-pCymRC-rJ/Cuma',
        'P67': 'pLux2-rJ/AHL',
        'P68': 'pLuxB-rJ/AHL',
        'P69': 'pPhlF-rJ/DAPG',
        'P70': 'pSalTTC-rJ/Sal',
        'P71': 'pVanCC-rJ/Van',
        'P72': 'rhaS-pRhaB-rJ/Rha',
        'P72': 'araC-pBAD/Ara',
        'E720': 'e11x33',
        'E721': 'e15x33',
        'E722': 'e16x33',
        'E723': 'e20x33',
        'E724': 'e32x33',
        'E725': 'e34x33',
        'E726': 'e38x33',
        'E727': 'e41x33',
        'E728': 'e42x33',
        'G726': 'e11x32STPhoRadA',
        'G721': 'e15x32NpuSspS2',
        'G722': 'e16x33NrdA2',
        'G723': 'e20x32gp411',
        'G724': 'e32x30SspGyrB',
        'G725': 'e34x30MjaKlbA',
        'G726': 'e38x32gp418',
        'G727': 'e41x32NrdJ1',
        'G728': 'e42x32STIMPDH1',
        'BK': 'blank-kan',
        'BA': 'blank-amp',
        '3K3-N': 'negative-control-kan',
        '4A3-N': 'negative-control-amp',
        '4A3-P': 'positive-control-amp'
    }
    #sample_map.update(control_map)
    df = pd.merge(well, plate_map, on='Well', how='left').dropna(subset=['value']).reset_index(drop=True)
    df['short_name'] = df['value'].map(sample_map) + '_' + df['suffix'].astype(int).astype(str)
    return df.dropna()

In [13]:
folder = '023-intact-vs-split-ecfs-rep2'
plate_map = read_map(pd.read_csv('datasets/experiment/{}/plate_map.csv'.format(folder)))
raw_data = pd.read_csv('datasets/experiment/{}/raw.csv'.format(folder), skiprows=[0])
metadata = generate_metadata(raw_data['Well'], plate_map)
data = raw_data[raw_data['Well'].isin(metadata['Well'])].reset_index(drop=True)
datas = generate_data(data, metadata['short_name'], h=22, m=40, num_data=4)

In [9]:
datas[0].to_csv('datasets/experiment/{}/ods.csv'.format(folder))
datas[1].to_csv('datasets/experiment/{}/fluos.csv'.format(folder))
datas[2].to_csv('datasets/experiment/{}/fluos-lower-1.csv'.format(folder))
datas[3].to_csv('datasets/experiment/{}/fluos-lower-2.csv'.format(folder))

In [14]:
datas[1]

short_name,e16x33_0,e16x33_1,e16x33_2,e16x33_3,e16x33_4,e16x33_5,e16x33NrdA2_0,e16x33NrdA2_1,e16x33NrdA2_2,e16x33NrdA2_3,e16x33NrdA2_4,e16x33NrdA2_5,e20x33_0,e20x33_1,e20x33_2,e20x33_3,e20x33_4,e20x33_5,e20x32gp411_0,e20x32gp411_1,e20x32gp411_2,e20x32gp411_3,e20x32gp411_4,e20x32gp411_5,e32x33_0,e32x33_1,e32x33_2,e32x33_3,e32x33_4,e32x33_5,e32x30SspGyrB_0,e32x30SspGyrB_1,e32x30SspGyrB_2,e32x30SspGyrB_3,e32x30SspGyrB_4,e32x30SspGyrB_5,e34x33_0,e34x33_1,e34x33_2,e34x33_3,e34x33_4,e34x33_5,e34x30MjaKlbA_0,e34x30MjaKlbA_1,e34x30MjaKlbA_2,e34x30MjaKlbA_3,e34x30MjaKlbA_4,e34x30MjaKlbA_5,e38x33_0,e38x33_1,e38x33_2,e38x33_3,e38x33_4,e38x33_5,e38x32gp418_0,e38x32gp418_1,e38x32gp418_2,e38x32gp418_3,e38x32gp418_4,e38x32gp418_5,e41x33_0,e41x33_1,e41x33_2,e41x33_3,e41x33_4,e41x33_5,e41x32NrdJ1_0,e41x32NrdJ1_1,e41x32NrdJ1_2,e41x32NrdJ1_3,e41x32NrdJ1_4,e41x32NrdJ1_5,e42x33_0,e42x33_1,e42x33_2,e42x33_3,e42x33_4,e42x33_5,e42x32STIMPDH1_0,e42x32STIMPDH1_1,e42x32STIMPDH1_2,e42x32STIMPDH1_3,e42x32STIMPDH1_4,e42x32STIMPDH1_5,blank-kan_0,blank-kan_1,negative-control-kan_0,negative-control-kan_1,blank-amp_0,blank-amp_1,negative-control-amp_0,negative-control-amp_1,positive-control-amp_0,positive-control-amp_1
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,25.0,0.0,35.0,0.0,28.0,3.0,19.0,8.0,12.0,-1.0,-8.0,-13.0,147.0,141.0,178.0,118.0,124.0,125.0,98.0,110.0,116.0,83.0,95.0,108.0,115.0,138.0,123.0,134.0,116.0,130.0,117.0,117.0,141.0,117.0,121.0,92.0,69.0,95.0,99.0,106.0,104.0,117.0,72.0,78.0,90.0,77.0,73.0,61.0,78.0,154.0,80.0,104.0,66.0,91.0,98.0,27.0,63.0,47.0,41.0,84.0,69.0,116.0,87.0,70.0,71.0,71.0,78.0,34.0,66.0,66.0,66.0,26.0,43.0,52.0,56.0,71.0,43.0,59.0,57.0,35.0,46.0,41.0,44.0,38.0,NaN,NaN,40.0,-32.0,NaN,NaN,54.0,32.0,1661.0,2080.0
20,11.0,16.0,9.0,15.0,7.0,-7.0,-5.0,-4.0,-11.0,-9.0,-10.0,-17.0,57.0,66.0,44.0,49.0,63.0,52.0,38.0,19.0,12.0,18.0,42.0,32.0,49.0,70.0,85.0,51.0,57.0,81.0,84.0,50.0,55.0,60.0,62.0,43.0,55.0,57.0,51.0,59.0,53.0,79.0,48.0,31.0,45.0,8.0,13.0,15.0,54.0,66.0,70.0,65.0,54.0,47.0,49.0,31.0,34.0,16.0,-2.0,44.0,57.0,57.0,56.0,30.0,41.0,59.0,39.0,53.0,42.0,36.0,43.0,36.0,25.0,29.0,12.0,53.0,15.0,26.0,24.0,11.0,15.0,18.0,-5.0,10.0,NaN,NaN,60.0,-21.0,NaN,NaN,80.0,42.0,1191.0,1383.0
40,20.5,-0.5,-2.5,13.5,-14.5,-19.5,-13.5,-18.5,-0.5,-16.5,-12.5,-0.5,77.5,51.5,29.5,32.5,42.5,31.5,43.5,20.5,21.5,10.5,20.5,21.5,36.5,68.5,67.5,51.5,32.5,55.5,56.5,47.5,32.5,42.5,48.5,34.5,34.5,51.5,48.5,41.5,32.5,47.5,15.5,27.5,39.5,12.5,20.5,3.5,67.5,61.5,34.5,27.5,30.5,37.5,32.5,14.5,14.5,-8.5,-10.5,59.5,62.5,64.5,52.5,3.5,44.5,66.5,38.5,15.5,35.5,45.5,6.5,16.5,36.5,21.5,39.5,39.5,29.5,15.5,35.5,9.5,6.5,7.5,14.5,-4.5,NaN,NaN,50.5,-17.5,NaN,NaN,68.0,54.0,1598.0,1827.0
60,43.0,35.0,23.0,17.0,16.0,22.0,-13.0,-8.0,-2.0,0.0,-4.0,6.0,94.0,69.0,36.0,44.0,28.0,24.0,34.0,45.0,11.0,28.0,25.0,22.0,73.0,60.0,57.0,39.0,46.0,40.0,55.0,55.0,58.0,48.0,61.0,57.0,40.0,36.0,40.0,32.0,59.0,58.0,11.0,11.0,16.0,5.0,20.0,11.0,81.0,86.0,49.0,54.0,56.0,53.0,36.0,23.0,7.0,27.0,-11.0,44.0,78.0,78.0,73.0,47.0,56.0,65.0,25.0,22.0,42.0,32.0,20.0,31.0,56.0,40.0,31.0,43.0,55.0,36.0,28.0,8.0,11.0,28.0,13.0,20.0,NaN,NaN,45.0,-4.0,NaN,NaN,59.0,66.0,2157.0,2479.0
80,57.0,38.0,21.0,5.0,-7.0,-8.0,2.0,1.0,-10.0,-12.0,6.0,22.0,91.0,70.0,55.0,39.0,62.0,44.0,25.0,37.0,13.0,21.0,17.0,38.0,80.0,79.0,73.0,71.0,52.0,62.0,46.0,48.0,54.0,36.0,61.0,48.0,48.0,62.0,54.0,47.0,35.0,56.0,-3.0,13.0,17.0,3.0,23.0,10.0,70.0,97.0,59.0,40.0,51.0,34.0,18.0,16.0,16.0,13.0,3.0,42.0,90.0,60.0,50.0,24.0,61.0,53.0,31.0,42.0,57.0,40.0,43.0,29.0,50.0,38.0,19.0,59.0,43.0,44.0,39.0,-9.0,25.0,13.0,30.0,7.0,NaN,NaN,37.0,-19.0,NaN,NaN,91.0,56.0,2721.0,3117.0
100,63.5,52.5,55.5,48.5,48.5,47.5,20.5,9.5,46.5,24.5,30.5,22.5,111.5,121.5,81.5,65.5,85.5,81.5,59.5,51.5,57.5,38.5,40.5,73.5,88.5,92.5,93.5,101.5,78.5,95.5,79.5,86.5,69.5,86.5,99.5,92.5,73.5,95.5,90.5,80.5,75.5,86.5,42.5,42.5,60.5,37.5,42.5,52.5,122.5,105.5,76.5,54.5,67.5,71.5,67.5,50.5,37.5,46.5,34.5,88.5,112.